In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Self Collect Dataset without pre processing**

In [ ]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Label/train"
data_dir_test = "/content/drive/MyDrive/Label/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2}

IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

# Load images without preprocessing
def load_data(data_dir):
    X = []  # Features (raw images)
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image to maintain consistent dimensions
            img = cv2.resize(img, IMG_SIZE)
            img = img.flatten()  # Convert to 1D vector

            X.append(img)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data(data_dir_test)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.5961538461538461
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.23      0.38        13
           1       0.68      0.68      0.68        22
           2       0.48      0.76      0.59        17

    accuracy                           0.60        52
   macro avg       0.72      0.56      0.55        52
weighted avg       0.70      0.60      0.58        52



# **Self Collect Dataset with pre processing**

In [ ]:
import os
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Label/train"
data_dir_test = "/content/drive/MyDrive/Label/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            color_features = extract_color_histogram(img)
            texture_features = extract_texture_features(img)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.8461538461538461
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.38      0.56        13
           1       0.73      1.00      0.85        22
           2       1.00      1.00      1.00        17

    accuracy                           0.85        52
   macro avg       0.91      0.79      0.80        52
weighted avg       0.89      0.85      0.82        52



# **Self Collect Dataset with pre processing (img_sharpen)**

In [5]:
import os
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Label/train"
data_dir_test = "/content/drive/MyDrive/Label/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

def sharpen_image(img):
    # Create the sharpening kernel
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    # Sharpen the image
    sharpened_image = cv2.filter2D(img, -1, kernel)
    return sharpened_image

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            img_sharpen = sharpen_image(img)
            color_features = extract_color_histogram(img_sharpen)
            texture_features = extract_texture_features(img_sharpen)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.8076923076923077
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.23      0.38        13
           1       0.69      1.00      0.81        22
           2       1.00      1.00      1.00        17

    accuracy                           0.81        52
   macro avg       0.90      0.74      0.73        52
weighted avg       0.87      0.81      0.77        52



# **Self Collect Dataset with pre processing (img_enhance)**

In [6]:
import os
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Label/train"
data_dir_test = "/content/drive/MyDrive/Label/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

def enhance_image(img):
    enhance_img = cv2.convertScaleAbs(img, alpha=1.5, beta=20)
    return enhance_img

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            img_enhance = enhance_image(img)
            color_features = extract_color_histogram(img_enhance)
            texture_features = extract_texture_features(img_enhance)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.9807692307692307
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       0.96      1.00      0.98        22
           2       1.00      1.00      1.00        17

    accuracy                           0.98        52
   macro avg       0.99      0.97      0.98        52
weighted avg       0.98      0.98      0.98        52



# **Self Collect Dataset with pre processing (remove_bg)**

In [7]:
import os
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Label/train"
data_dir_test = "/content/drive/MyDrive/Label/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

def remove_background(img):
    # Convert to grayscale and apply binary thresholding
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Create mask for grabCut
    mask = np.zeros(img.shape[:2], np.uint8)
    mask[binary_mask == 255] = cv2.GC_PR_BGD
    mask[binary_mask == 0] = cv2.GC_PR_FGD

    # Define models
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Run grabCut with the new mask
    cv2.grabCut(img, mask, None, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_MASK)

    # Modify mask to extract foreground
    mask2 = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD), 0, 1).astype('uint8')
    img_bg_removed = img * mask2[:, :, np.newaxis]

    return img_bg_removed

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            remove_background_img = remove_background(img)
            color_features = extract_color_histogram(remove_background_img)
            texture_features = extract_texture_features(remove_background_img)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.8269230769230769
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.31      0.47        13
           1       0.73      1.00      0.85        22
           2       0.94      1.00      0.97        17

    accuracy                           0.83        52
   macro avg       0.89      0.77      0.76        52
weighted avg       0.87      0.83      0.79        52



# **Public Dataset without pre processing**

In [3]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/train"
data_dir_test = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2, "Level4": 3, "Level5": 4}

IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

# Load images without preprocessing
def load_data(data_dir):
    X = []  # Features (raw images)
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image to maintain consistent dimensions
            img = cv2.resize(img, IMG_SIZE)
            img = img.flatten()  # Convert to 1D vector

            X.append(img)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data(data_dir_test)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.6691576086956522
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.83      0.83      1071
           1       0.55      0.56      0.55       669
           2       0.57      0.62      0.59       827
           3       0.61      0.62      0.61       989
           4       0.74      0.66      0.70       860

    accuracy                           0.67      4416
   macro avg       0.66      0.66      0.66      4416
weighted avg       0.67      0.67      0.67      4416



# **Public Dataset with pre processing**

In [4]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/train"
data_dir_test = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2, "Level4": 3, "Level5": 4}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            color_features = extract_color_histogram(img)
            texture_features = extract_texture_features(img)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.7296195652173914
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1071
           1       0.73      0.64      0.68       669
           2       0.70      0.72      0.71       827
           3       0.62      0.68      0.65       989
           4       0.71      0.65      0.68       860

    accuracy                           0.73      4416
   macro avg       0.73      0.72      0.72      4416
weighted avg       0.73      0.73      0.73      4416



# **Public Dataset with pre processing (img_sharpen)**

In [4]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/train"
data_dir_test = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2, "Level4": 3, "Level5": 4}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

def sharpen_image(img):
    # Create the sharpening kernel
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    # Sharpen the image
    sharpened_image = cv2.filter2D(img, -1, kernel)
    return sharpened_image

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            img_sharpen = sharpen_image(img)
            color_features = extract_color_histogram(img_sharpen)
            texture_features = extract_texture_features(img_sharpen)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.7284873188405797
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.92      0.88      1071
           1       0.72      0.59      0.65       669
           2       0.68      0.72      0.70       827
           3       0.62      0.67      0.64       989
           4       0.76      0.68      0.71       860

    accuracy                           0.73      4416
   macro avg       0.73      0.71      0.72      4416
weighted avg       0.73      0.73      0.73      4416



# **Public Dataset with pre processing (img_enhance)**

In [5]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/train"
data_dir_test = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2, "Level4": 3, "Level5": 4}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

def enhance_image(img):
    enhance_img = cv2.convertScaleAbs(img, alpha=1.5, beta=20)
    return enhance_img

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            img_enhance = enhance_image(img)
            color_features = extract_color_histogram(img_enhance)
            texture_features = extract_texture_features(img_enhance)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.6788949275362319
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1071
           1       0.72      0.60      0.65       669
           2       0.65      0.67      0.66       827
           3       0.53      0.63      0.58       989
           4       0.63      0.52      0.57       860

    accuracy                           0.68      4416
   macro avg       0.68      0.67      0.67      4416
weighted avg       0.68      0.68      0.68      4416



# **Public Dataset with pre processing (remove_bg)**

In [6]:
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Set paths
data_dir_train = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/train"
data_dir_test = "/content/drive/MyDrive/Hass Avocado Ripening Photographic Dataset/outfolder/test"
labels_dict = {"Level1": 0, "Level2": 1, "Level3": 2, "Level4": 3, "Level5": 4}

# Feature extraction parameters
COLOR_BINS = 32
IMG_SIZE = (64, 64)  # Maintain resizing only for consistent input dimensions

def remove_background(img):
    # Convert to grayscale and apply binary thresholding
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Create mask for grabCut
    mask = np.zeros(img.shape[:2], np.uint8)
    mask[binary_mask == 255] = cv2.GC_PR_BGD
    mask[binary_mask == 0] = cv2.GC_PR_FGD

    # Define models
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Run grabCut with the new mask
    cv2.grabCut(img, mask, None, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_MASK)

    # Modify mask to extract foreground
    mask2 = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD), 0, 1).astype('uint8')
    img_bg_removed = img * mask2[:, :, np.newaxis]

    return img_bg_removed

# Extract color histogram features
def extract_color_histogram(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [COLOR_BINS, COLOR_BINS, COLOR_BINS], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Placeholder for texture feature extraction (can be extended)
def extract_texture_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img.flatten()[:COLOR_BINS]  # Simplified feature example

# Load images and extract features
def load_data_and_extract_features(data_dir):
    X = []  # Features
    y = []  # Labels

    for label_name, label_id in labels_dict.items():
        class_dir = os.path.join(data_dir, label_name)
        if not os.path.exists(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize image
            img = cv2.resize(img, IMG_SIZE)

            # Extract features
            remove_background_img = remove_background(img)
            color_features = extract_color_histogram(remove_background_img)
            texture_features = extract_texture_features(remove_background_img)
            features = np.concatenate((color_features, texture_features))

            X.append(features)
            y.append(label_id)

    return np.array(X), np.array(y)

# Load the train and test data
print("Loading training data with resizing...")
X_train, y_train = load_data_and_extract_features(data_dir_train)
print("Loading testing data with resizing...")
X_test, y_test = load_data_and_extract_features(data_dir_test)

# Normalize features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust components as needed
X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

# Create and train SVM model
print("Training the SVM model with resizing...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train)

# Predictions
print("Making predictions with resizing...")
y_pred = svm_model.predict(X_test_pca)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Loading training data with resizing...
Loading testing data with resizing...
Training the SVM model with resizing...
Making predictions with resizing...
Accuracy: 0.6748188405797102
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1071
           1       0.71      0.59      0.65       669
           2       0.65      0.67      0.66       827
           3       0.53      0.61      0.56       989
           4       0.62      0.52      0.57       860

    accuracy                           0.67      4416
   macro avg       0.67      0.66      0.67      4416
weighted avg       0.68      0.67      0.67      4416

